In [72]:
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras import callbacks
import os
import cv2
import string
import numpy as np
from scipy import stats



symbols = string.ascii_letters + "0123456789"
num_symbols = len(symbols)
img_shape =(50,200,1)

In [73]:
def create_net():
    img = layers.Input(shape=img_shape)
    conv1 = layers.Conv2D(16,(3,3),padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)
    conv2 = layers.Conv2D(32,(3,3),padding='same',activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)
    conv3 = layers.Conv2D(32,(3,3),padding='same',activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPool2D(padding='same')(bn)

    flat = layers.Flatten()(mp3)
    outs=[]
    for _ in range(5):
        dens1 = layers.Dense(64,activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols,activation='sigmoid')(drop)

        outs.append(res)
    model = Model(img,outs)
    model.compile(optimizer='rmsprop', loss=['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'],metrics=['accuracy'])
    return model
        

In [74]:
def signaltonoise(a, axis=None, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)

In [75]:
def preprocess_data():
    n_samples = len(os.listdir(r'C:\Users\sunda\OneDrive\Documents\Jupyter\samples'))
    x = np.zeros((n_samples,50,200,1))
    y = np.zeros((5,n_samples,num_symbols))
    z= np.zeros((n_samples,1))
    for i, pic in enumerate(os.listdir(r'C:\Users\sunda\OneDrive\Documents\Jupyter\samples')):
        img = cv2.imread(os.path.join(r'C:\Users\sunda\OneDrive\Documents\Jupyter\samples',pic),cv2.IMREAD_GRAYSCALE)
        pic_target = pic[:-4]
        snr = signaltonoise(img,axis=None)
        if len(pic_target)<6:
            img = img/255.0
            img = np.reshape(img,(50,200,1))

            targs = np.zeros((5,num_symbols))
            for j,l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j,ind]=1
            x[i] = img
            y[:,i] = targs
            z[i]= snr
    return x,y,z
x,y,z = preprocess_data()
print(x[971])
print(z[971])
x_train,y_train = x[:970],y[:,:970]
x_test,y_test = x[970:],y[:,970:]


[[[0.75294118]
  [0.75294118]
  [0.75294118]
  ...
  [0.98431373]
  [0.98431373]
  [0.98431373]]

 [[0.75294118]
  [0.75294118]
  [0.75294118]
  ...
  [0.98431373]
  [0.98431373]
  [0.98431373]]

 [[0.75294118]
  [0.75294118]
  [0.75294118]
  ...
  [0.98431373]
  [0.98431373]
  [0.98431373]]

 ...

 [[0.76470588]
  [0.76470588]
  [0.76470588]
  ...
  [0.99607843]
  [0.99607843]
  [0.99607843]]

 [[0.76470588]
  [0.76470588]
  [0.76470588]
  ...
  [0.99607843]
  [0.99607843]
  [0.99607843]]

 [[0.76470588]
  [0.76470588]
  [0.76470588]
  ...
  [0.99607843]
  [0.99607843]
  [0.99607843]]]
[2.34598957]


In [76]:
net = create_net()
history = net.fit(x_train,[y_train[0],y_train[1],y_train[2],y_train[3],y_train[4]],batch_size=64,epochs=25,validation_split=0.2)

Epoch 1/25
13/13 [==============================] - 4s 286ms/step - loss: 19.5596 - dense_101_loss: 3.9362 - dense_103_loss: 3.8796 - dense_105_loss: 3.8606 - dense_107_loss: 4.0024 - dense_109_loss: 3.8808 - dense_101_accuracy: 0.0361 - dense_103_accuracy: 0.0606 - dense_105_accuracy: 0.0451 - dense_107_accuracy: 0.0232 - dense_109_accuracy: 0.0309 - val_loss: 19.9264 - val_dense_101_loss: 4.1205 - val_dense_103_loss: 3.9324 - val_dense_105_loss: 3.9506 - val_dense_107_loss: 3.9822 - val_dense_109_loss: 3.9406 - val_dense_101_accuracy: 0.0000e+00 - val_dense_103_accuracy: 0.0258 - val_dense_105_accuracy: 0.0515 - val_dense_107_accuracy: 0.0515 - val_dense_109_accuracy: 0.0258
Epoch 2/25
13/13 [==============================] - 3s 243ms/step - loss: 17.5539 - dense_101_loss: 3.4489 - dense_103_loss: 3.4293 - dense_105_loss: 3.5114 - dense_107_loss: 3.6124 - dense_109_loss: 3.5519 - dense_101_accuracy: 0.0760 - dense_103_accuracy: 0.0593 - dense_105_accuracy: 0.0657 - dense_107_accuracy

In [77]:
def predict(filepath):
    img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)/255.0
    res = np.array(net.predict(img[np.newaxis,:,:,np.newaxis]))
    ans = np.reshape(res,(5,62))
    l_ind=[]
    for a in ans:
        l_ind.append(np.argmax(a))
    capt=''
    for l in l_ind:
        capt+=symbols[l]
    return capt

In [78]:
import matplotlib.pyplot as plt
plt.subplot(211)
plt.title('Loss')
plt.plot(history.history['loss'],label='train')
plt.plot(history.history['val_loss'],label='test')
plt.legend()
plt.subplot(212)
plt.title('Accuracy')
plt.plot(history.history['dense_91_categorical_accuracy'],label='train')
plt.plot(history.history['val_dense_91_categorical_accuracy'],label='test')
plt.legend()
history_dict = history.history
print(history_dict.keys())

KeyError: 'dense_91_categorical_accuracy'

In [51]:
net.evaluate(x_test,[y_test[0],y_test[1],y_test[2],y_test[3],y_test[4]])

print(predict(r'C:\Users\sunda\OneDrive\Documents\Jupyter\samples\5mcy7.png'))
#print(predict(r'C:\Users\sunda\OneDrive\Documents\Jupyter\sampl`es\5np4m.png'))
print(predict(r'C:\Users\sunda\OneDrive\Documents\Jupyter\samples\56m6y.png'))

4/4 [==============================] - 0s 20ms/step - loss: 10.4769 - dense_61_loss: 5.5292 - dense_63_loss: 1.0044 - dense_65_loss: 1.3037 - dense_67_loss: 1.4679 - dense_69_loss: 1.1716 - dense_61_categorical_accuracy: 0.0000e+00 - dense_63_categorical_accuracy: 0.9200 - dense_65_categorical_accuracy: 0.7900 - dense_67_categorical_accuracy: 0.7000 - dense_69_categorical_accuracy: 0.7300
5mcy7
56m6y


In [ ]:
#from sklearn.naive_bayes import MultinomialNB
#modelmnb = MultinomialNB()
#modelmnb()